In [2]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LeakyReLU, Input, concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l1_l2


import glob
import os

from scipy.stats import zscore
from functools import reduce

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
weather_path = os.path.join('..','data','weather','precip_temp.csv')
weather_df = pd.read_csv(weather_path)
clients_path = os.path.join('..','data','wifi','**','Clients per day.csv')
clients_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                           glob.glob(clients_path)), ignore_index=True)
sessions_path = os.path.join('..','data','wifi','**','Number of sessions over time.csv')
sessions_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                               glob.glob(sessions_path)), ignore_index=True)
usage_path = os.path.join('..','data','wifi','**','Usage over time.csv')
usage_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                               glob.glob(usage_path)), ignore_index=True)
# Interpolate zeros in usage data
usage_df.loc[usage_df['Total (B)'] == 0, 'Total (B)'] = np.NaN
usage_df['Total (B)'] = usage_df['Total (B)'].interpolate()
usage_df.at[0, 'Total (B)'] = usage_df.loc[1]['Total (B)']

In [11]:
# Add day of the week as a one-hot
weekday_labels = []
for d in range(7):
    label = 'weekday-%i' % d
    weekday_labels.append(label)
    clients_df[label] = [int(dt.weekday()==d) for dt in clients_df['Time']]
    
# Put wifi date in the same format as it is in the weather data
clients_df['Time'] = [dt.strftime('%Y-%m-%d') for dt in clients_df['Time']]
sessions_df['Time'] = [dt.strftime('%Y-%m-%d') for dt in sessions_df['Time']]
usage_df['Date'] = [dt.strftime('%Y-%m-%d') for dt in usage_df['Time']]
usage_df['Hour'] = [dt.hour for dt in usage_df['Time']]


all_data = clients_df.merge(weather_df, left_on='Time', right_on='DATE') \
    .merge(sessions_df, left_on='Time', right_on='Time')

# Put 4-hour chunks together into rows by day
usage_labels = set()
for index, row in usage_df.iterrows():
    total_label = 'total-%i' % row['Hour']
    all_data.loc[all_data['DATE'] == row['Date'], total_label] \
        = row['Total (B)']
    usage_labels.add(total_label)
usage_labels = list(usage_labels)
# Normalize some inputs using z-scores
cols_to_norm = ['TMIN', 'TMAX', 'PRCP']
for col in cols_to_norm:
    all_data[col] = zscore(all_data[col])

# Separate inputs into categories for encoding
day_x = all_data[weekday_labels].values
weather_x = all_data[['PRCP', 'TMAX', 'TMIN']].values
date_x = np.expand_dims(np.arange(0, 353/365, 1/365).astype('float32'), axis=1)

y_labels = ['# Clients'] + usage_labels
y = all_data[y_labels].values

clients_y=all_data[['# Clients']].values
usage_y=all_data[usage_labels].values
sessions_y=all_data[['# Sessions']].values

all_data

,Time,# Clients,weekday-0,weekday-1,weekday-2,weekday-3,weekday-4,weekday-5,weekday-6,DATE,PRCP,TMAX,TMIN,# Sessions,total-22,total-2,total-6,total-10,total-14,total-18
0,2017-01-01,93,0,0,0,0,0,0,1,2017-01-01,-0.343154,-0.820006,-0.792066,0,1.951482e+05,52117.617780,243.484444,4.312001e+05,1.795721e+05,1.372848e+05
1,2017-01-02,73,1,0,0,0,0,0,0,2017-01-02,0.579624,-1.667506,-0.792066,0,4.505202e+04,44.942222,7420.586667,7.166350e+04,2.816171e+05,2.650556e+05
2,2017-01-03,87,0,1,0,0,0,0,0,2017-01-03,1.589692,-1.067194,-0.792066,0,1.047097e+04,230941.582200,461.937778,1.435540e+05,4.288307e+05,6.002688e+04
3,2017-01-04,97,0,0,1,0,0,0,0,2017-01-04,-0.343154,-0.725840,-1.262831,0,1.264754e+04,1755.022222,7644.728889,3.313021e+05,1.094120e+06,1.694071e+05
4,2017-01-05,92,0,0,0,1,0,0,0,2017-01-05,-0.343154,-2.138339,-1.427599,0,1.624292e+04,3751.822222,785.066667,5.481188e+04,2.631856e+05,2.472465e+05
5,2017-01-06,88,0,0,0,0,1,0,0,2017-01-06,-0.218454,-2.208964,-1.757135,0,2.263950e+04,8192.568891,142.222222,1.798542e+04,1.443812e+05,3.356718e+05
6,2017-01-07,39,0,0,0,0,0,1,0,2017-01-07,0.592094,-2.832818,-2.580974,0,1.137778e+03,490.951111,2421.760000,1.387463e+04,2.165817e+05,4.313145e+04
7,2017-01-08,40,0,0,0,0,0,0,1,2017-01-08,-0.318214,-2.997609,-2.828126,0,1.647502e+03,1137.777778,1476.835556,1.591637e+04,9.978994e+04,1.923584e+04
8,2017-01-09,74,1,0,0,0,0,0,0,2017-01-09,-0.343154,-2.950526,-2.710435,0,2.526436e+03,87.608889,7751.111111,3.114325e+04,5.712549e+05,2.630582e+05
9,2017-01-10,67,0,1,0,0,0,0,0,2017-01-10,-0.343154,-2.020631,-2.322053,0,4.384996e+03,1701.262222,876.088889,1.000078e+06,9.430374e+05,4.075776e+05


In [16]:
epochs = 1000
reg = None # l1_l2(0.01,0.02)

def stack_layers(layers):
    return reduce(lambda stack, e: e(stack), layers)

weather_layers = \
  [Input((3,))] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(5, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(4, activation='relu',kernel_regularizer=reg)]

weather_out = stack_layers(weather_layers)

day_layers = \
  [Input((7,))] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(5, activation='relu',kernel_regularizer=reg)]

day_out = stack_layers(day_layers)

date_layers = \
  [Input((1,))] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(5, activation='relu',kernel_regularizer=reg)]

date_out = stack_layers(date_layers)

main_layers = \
  [concatenate([weather_out, day_out, date_out])] \
+ [Dense(20, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(20, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(20, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] 

main_out = stack_layers(main_layers)

usage_layers = \
  [main_out] \
+ [Dense(15, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(6, activation='relu',kernel_regularizer=reg)]

usage_out=stack_layers(usage_layers)

clients_layers = \
  [main_out] \
+ [Dense(15, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(1, activation='relu',kernel_regularizer=reg)]

clients_out = stack_layers(clients_layers)

sessions_layers = \
  [main_out] \
+ [Dense(15, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(10, activation='relu',kernel_regularizer=reg)] \
+ [Dropout(0.5)] \
+ [Dense(1, activation='relu',kernel_regularizer=reg)]

sessions_out = stack_layers(sessions_layers)

model = Model(inputs=[weather_layers[0], day_layers[0], date_layers[0]], outputs=[usage_out,clients_out,sessions_out])
model.compile(loss='mean_absolute_percentage_error',
              optimizer=Adam(0.0001))

model.summary()
print(usage_y.shape)
print(clients_y.shape)
print(sessions_y.shape)
early_stopping = EarlyStopping(patience=50, verbose=1)
history = model.fit([weather_x,day_x,date_x], [usage_y,clients_y,sessions_y],
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 7)            0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
dense_109 (Dense)               (None, 10)           40          input_13[0][0]                   
__________________________________________________________________________________________________
dense_113 

Train on 317 samples, validate on 36 samples
Epoch 1/1000
317/317 [==============================] - 1s 4ms/step - loss: 8556480.1735 - dense_127_loss: 99.9987 - dense_131_loss: 99.7188 - dense_135_loss: 8556280.5484 - val_loss: 299.6167 - val_dense_127_loss: 100.0000 - val_dense_131_loss: 99.6167 - val_dense_135_loss: 100.0000
Epoch 2/1000
317/317 [==============================] - 0s 82us/step - loss: 2325247.5988 - dense_127_loss: 99.9983 - dense_131_loss: 99.8613 - dense_135_loss: 2325047.7190 - val_loss: 299.6023 - val_dense_127_loss: 100.0000 - val_dense_131_loss: 99.6023 - val_dense_135_loss: 100.0000
Epoch 3/1000
317/317 [==============================] - 0s 90us/step - loss: 1047868.6598 - dense_127_loss: 99.9583 - dense_131_loss: 99.5788 - dense_135_loss: 1047669.1083 - val_loss: 299.5845 - val_dense_127_loss: 100.0000 - val_dense_131_loss: 99.5845 - val_dense_135_loss: 100.0000
Epoch 4/1000
317/317 [==============================] - 0s 85us/step - loss: 6005124.4821 - dense_

317/317 [==============================] - 0s 85us/step - loss: 1204226.4907 - dense_127_loss: 99.9916 - dense_131_loss: 99.3740 - dense_135_loss: 1204027.1744 - val_loss: 299.2223 - val_dense_127_loss: 99.9993 - val_dense_131_loss: 99.2230 - val_dense_135_loss: 100.0000
Epoch 30/1000
317/317 [==============================] - 0s 76us/step - loss: 4203377.3140 - dense_127_loss: 99.9945 - dense_131_loss: 99.4526 - dense_135_loss: 4203177.8649 - val_loss: 299.2066 - val_dense_127_loss: 99.9993 - val_dense_131_loss: 99.2073 - val_dense_135_loss: 100.0000
Epoch 31/1000
317/317 [==============================] - 0s 76us/step - loss: 4841136.9069 - dense_127_loss: 99.9932 - dense_131_loss: 99.5633 - dense_135_loss: 4840937.2725 - val_loss: 299.1916 - val_dense_127_loss: 99.9993 - val_dense_131_loss: 99.1923 - val_dense_135_loss: 100.0000
Epoch 32/1000
317/317 [==============================] - 0s 70us/step - loss: 2836901.8368 - dense_127_loss: 99.9939 - dense_131_loss: 99.4824 - dense_135_l

Epoch 58/1000
317/317 [==============================] - 0s 77us/step - loss: 17970104.9374 - dense_127_loss: 99.9905 - dense_131_loss: 99.2939 - dense_135_loss: 17969906.4834 - val_loss: 298.7367 - val_dense_127_loss: 99.9918 - val_dense_131_loss: 98.7449 - val_dense_135_loss: 100.0000
Epoch 59/1000
317/317 [==============================] - 0s 76us/step - loss: 3854085.4853 - dense_127_loss: 99.9941 - dense_131_loss: 98.8186 - dense_135_loss: 3853886.6363 - val_loss: 298.7203 - val_dense_127_loss: 99.9908 - val_dense_131_loss: 98.7295 - val_dense_135_loss: 100.0000
Epoch 60/1000
317/317 [==============================] - 0s 76us/step - loss: 11165627.3155 - dense_127_loss: 99.9671 - dense_131_loss: 98.9897 - dense_135_loss: 11165428.3002 - val_loss: 298.7033 - val_dense_127_loss: 99.9899 - val_dense_131_loss: 98.7133 - val_dense_135_loss: 100.0000
Epoch 61/1000
317/317 [==============================] - 0s 85us/step - loss: 1224754.1522 - dense_127_loss: 99.9826 - dense_131_loss: 99.

Epoch 87/1000
317/317 [==============================] - 0s 79us/step - loss: 288.6628 - dense_127_loss: 99.9911 - dense_131_loss: 98.7679 - dense_135_loss: 89.9038 - val_loss: 297.9772 - val_dense_127_loss: 99.9752 - val_dense_131_loss: 98.0020 - val_dense_135_loss: 100.0000
Epoch 88/1000
317/317 [==============================] - 0s 79us/step - loss: 8385380.1789 - dense_127_loss: 99.9759 - dense_131_loss: 98.6795 - dense_135_loss: 8385181.3733 - val_loss: 297.9419 - val_dense_127_loss: 99.9748 - val_dense_131_loss: 97.9671 - val_dense_135_loss: 100.0000
Epoch 89/1000
317/317 [==============================] - 0s 79us/step - loss: 3653090.7393 - dense_127_loss: 99.9761 - dense_131_loss: 98.5849 - dense_135_loss: 3652892.0858 - val_loss: 297.9016 - val_dense_127_loss: 99.9739 - val_dense_131_loss: 97.9277 - val_dense_135_loss: 100.0000
Epoch 90/1000
317/317 [==============================] - 0s 70us/step - loss: 5697292.7808 - dense_127_loss: 99.9864 - dense_131_loss: 98.7820 - dense_

...

317/317 [==============================] - 0s 78us/step - loss: 3439008.8609 - dense_127_loss: 99.8914 - dense_131_loss: 75.8188 - dense_135_loss: 3438832.9715 - val_loss: 259.3222 - val_dense_127_loss: 99.5644 - val_dense_131_loss: 59.7578 - val_dense_135_loss: 100.0000
Epoch 349/1000
317/317 [==============================] - 0s 81us/step - loss: 263.0122 - dense_127_loss: 99.9510 - dense_131_loss: 73.1561 - dense_135_loss: 89.9051 - val_loss: 259.1918 - val_dense_127_loss: 99.5633 - val_dense_131_loss: 59.6284 - val_dense_135_loss: 100.0000
Epoch 350/1000
317/317 [==============================] - 0s 79us/step - loss: 3768902.2159 - dense_127_loss: 99.9463 - dense_131_loss: 72.3808 - dense_135_loss: 3768730.0566 - val_loss: 259.0485 - val_dense_127_loss: 99.5623 - val_dense_131_loss: 59.4862 - val_dense_135_loss: 100.0000
Epoch 351/1000
317/317 [==============================] - 0s 70us/step - loss: 264.0598 - dense_127_loss: 99.8897 - dense_131_loss: 74.2652 - dense_135_loss: 89.90

Epoch 377/1000
317/317 [==============================] - 0s 87us/step - loss: 260.4207 - dense_127_loss: 99.9239 - dense_131_loss: 70.5915 - dense_135_loss: 89.9053 - val_loss: 256.0259 - val_dense_127_loss: 99.5250 - val_dense_131_loss: 56.5009 - val_dense_135_loss: 100.0000
Epoch 378/1000
317/317 [==============================] - 0s 76us/step - loss: 264.2028 - dense_127_loss: 99.9333 - dense_131_loss: 74.3642 - dense_135_loss: 89.9053 - val_loss: 255.9659 - val_dense_127_loss: 99.5232 - val_dense_131_loss: 56.4427 - val_dense_135_loss: 100.0000
Epoch 379/1000
317/317 [==============================] - 0s 76us/step - loss: 485029.0128 - dense_127_loss: 99.8538 - dense_131_loss: 66.2502 - dense_135_loss: 484862.9148 - val_loss: 255.8905 - val_dense_127_loss: 99.5211 - val_dense_131_loss: 56.3694 - val_dense_135_loss: 100.0000
Epoch 380/1000
317/317 [==============================] - 0s 77us/step - loss: 260.3320 - dense_127_loss: 99.9091 - dense_131_loss: 70.5176 - dense_135_loss: 8

317/317 [==============================] - 0s 73us/step - loss: 255.8095 - dense_127_loss: 99.9121 - dense_131_loss: 65.9924 - dense_135_loss: 89.9051 - val_loss: 253.5086 - val_dense_127_loss: 99.4813 - val_dense_131_loss: 54.0273 - val_dense_135_loss: 100.0000
Epoch 407/1000
317/317 [==============================] - 0s 76us/step - loss: 259.4748 - dense_127_loss: 99.9249 - dense_131_loss: 69.6446 - dense_135_loss: 89.9053 - val_loss: 253.4335 - val_dense_127_loss: 99.4793 - val_dense_131_loss: 53.9542 - val_dense_135_loss: 100.0000
Epoch 408/1000
317/317 [==============================] - 0s 73us/step - loss: 260.1331 - dense_127_loss: 99.9734 - dense_131_loss: 70.2544 - dense_135_loss: 89.9053 - val_loss: 253.3671 - val_dense_127_loss: 99.4785 - val_dense_131_loss: 53.8886 - val_dense_135_loss: 100.0000
Epoch 409/1000
317/317 [==============================] - 0s 92us/step - loss: 257.6598 - dense_127_loss: 99.9259 - dense_131_loss: 67.8286 - dense_135_loss: 89.9053 - val_loss: 253

317/317 [==============================] - 0s 70us/step - loss: 257.3616 - dense_127_loss: 99.9433 - dense_131_loss: 67.5130 - dense_135_loss: 89.9053 - val_loss: 252.6199 - val_dense_127_loss: 99.4587 - val_dense_131_loss: 53.1611 - val_dense_135_loss: 100.0000
Epoch 436/1000
317/317 [==============================] - 0s 73us/step - loss: 257.1779 - dense_127_loss: 99.9281 - dense_131_loss: 67.3445 - dense_135_loss: 89.9053 - val_loss: 252.6288 - val_dense_127_loss: 99.4575 - val_dense_131_loss: 53.1713 - val_dense_135_loss: 100.0000
Epoch 437/1000
317/317 [==============================] - 0s 66us/step - loss: 1047143.6672 - dense_127_loss: 99.8928 - dense_131_loss: 64.2950 - dense_135_loss: 1046979.4700 - val_loss: 252.6423 - val_dense_127_loss: 99.4559 - val_dense_131_loss: 53.1865 - val_dense_135_loss: 100.0000
Epoch 438/1000
317/317 [==============================] - 0s 70us/step - loss: 338113.3126 - dense_127_loss: 99.9083 - dense_131_loss: 63.8007 - dense_135_loss: 337949.6149

317/317 [==============================] - 0s 76us/step - loss: 860860.2076 - dense_127_loss: 99.8880 - dense_131_loss: 63.7425 - dense_135_loss: 860696.5552 - val_loss: 253.4190 - val_dense_127_loss: 99.4116 - val_dense_131_loss: 54.0074 - val_dense_135_loss: 100.0000
Epoch 465/1000
317/317 [==============================] - 0s 70us/step - loss: 254.8989 - dense_127_loss: 99.8852 - dense_131_loss: 65.1085 - dense_135_loss: 89.9051 - val_loss: 253.4449 - val_dense_127_loss: 99.4099 - val_dense_131_loss: 54.0351 - val_dense_135_loss: 100.0000
Epoch 466/1000
317/317 [==============================] - 0s 70us/step - loss: 257.6053 - dense_127_loss: 99.9005 - dense_131_loss: 67.7994 - dense_135_loss: 89.9054 - val_loss: 253.4799 - val_dense_127_loss: 99.4088 - val_dense_131_loss: 54.0711 - val_dense_135_loss: 100.0000
Epoch 467/1000
317/317 [==============================] - 0s 85us/step - loss: 254.5852 - dense_127_loss: 99.9342 - dense_131_loss: 64.7461 - dense_135_loss: 89.9050 - val_lo

In [ ]:
evalSeg=317
print (weather_x[evalSeg:].shape)
model.evaluate(x=[weather_x[evalSeg:],day_x[evalSeg:],date_x[evalSeg:]],y=[y[evalSeg:]])